In [ ]:
import pickle
with open(r"/content/images.p","rb") as f:
  images=pickle.load(f)
print(images.shape)
with open(r"/content/labels.p","rb") as f:
  labels=pickle.load(f)
print(labels.shape)
print("done")

In [ ]:
set(labels)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
set(labels)

In [ ]:
import numpy as np
n_persons = len(set(labels))
print("Number of persons: ", n_persons)
label_mapping = le.inverse_transform(np.arange(n_persons))
for i in range(len(label_mapping)):
  print(i, "-->", label_mapping[i])

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(images[20], cmap=plt.get_cmap("gray"))
plt.show()

In [ ]:
import cv2
def preprocessing(img):
  img = cv2.equalizeHist(img)
  img = img.reshape(100, 100, 1)
  img = img/255
  return img

In [ ]:
images = np.array(list(map(preprocessing, images)))
print("Shape of Input: ", images.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
labels = to_categorical(labels)

In [ ]:
categories = labels.shape[1]
print(categories)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
# import convolution layer
from tensorflow.keras.layers import Conv2D
# import pooling layer
from tensorflow.keras.layers import MaxPooling2D
# import faltten layer
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Conv2D(64, (5,5), input_shape=(100, 100, 1), activation="relu"))
model.add(Conv2D(64, (5,5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation="relu"))

model.add(Dense(4, activation="softmax"))
model.compile(Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
h=model.fit(images,labels,validation_split=0.2,batch_size=50,epochs=10,verbose=1)

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
layer0 = Model(model.layers[0].input, model.layers[0].output)
features = layer0.predict(images[69].reshape(1,100,100,1))

In [ ]:
features.shape

In [ ]:
plt.figure(figsize=(10,15))
for i in range(32):
  axes = plt.subplot(8, 4, i+1)
  plt.imshow(features[0,:,:,i])

In [ ]:
from google.colab import files
upload=files.upload()

In [ ]:
d=list(upload.keys())[0]

In [ ]:
import cv2
a=np.fromstring(upload[d],np.uint8)
img=cv2.imdecode(a,cv2.IMREAD_COLOR)
img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
print(a)
plt.imshow(img,cmap=plt.get_cmap("gray"))

In [ ]:
img=np.asarray(img)
img=cv2.resize(img,(100,100))
img=preprocessing(img)


In [ ]:
img=img.reshape(1,100,100,1)
print(model.predict(img))

In [ ]:
prediction=model.predict(img)

In [ ]:
p=np.argmax(prediction,axis=1)
p

In [ ]:
model.save('finalmodel.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')